In [ ]:
from importlib import reload
import sys
src_path =   '../src' # change as needed
sys.path.insert(0,src_path)

In [ ]:
import data_generator;reload(data_generator)

#Start session
max_length = 128

train_data, val_data, test_data = data_generator.GetData(max_length)

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import model_utils; reload(model_utils)

adam_customized = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.91, beta_2=0.999, epsilon=None, decay=0.1, amsgrad=False)
    
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

model = model_utils.NER()
    
model.generate(max_length, train_layers=4, optimizer = adam_customized, debias=False, debiasWeight=0.95)

# Instantiate variables
model_utils.initialize_vars(sess)

model.fit(
    train_data, 
    val_data,
    epochs=8,
    batch_size=32
)

In [ ]:
cm = model.score(test_data)

In [ ]:
bias = model.getBiasedPValues(test_data, num_iterations=10000)

In [ ]:
bias